In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection; python setup.py install

# 사용하는 cuda와 torch 버전에 맞게 mmdetection 다운

Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html
     |████████████████████████████████| 37.5 MB 178 kB/s 
     |████████████████████████████████| 190 kB 28.5 MB/s 
Cloning into 'mmdetection'...
remote: Enumerating objects: 24362, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 24362 (delta 3), reused 1 (delta 0), pack-reused 24348
Receiving objects: 100% (24362/24362), 37.49 MiB | 16.19 MiB/s, done.
Resolving deltas: 100% (17051/17051), done.
running install
running bdist_egg
running egg_info
creating mmdet.egg-info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
writing manifest file 'mmdet.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'mmdet.egg-info/SOURCES.

In [ ]:
# 런타임 재시작 필요

from mmdet.apis import init_detector, inference_detector
import mmcv

In [ ]:
%cd /content

/content


In [ ]:
%pip install wandb

     |████████████████████████████████| 1.8 MB 8.6 MB/s 
     |████████████████████████████████| 181 kB 48.7 MB/s 
     |████████████████████████████████| 144 kB 54.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=a3306900eea8a05b966fd0f84d4b2c16f4ba9c8592d12e4dc5844d951747b65b
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
!pip install wandb -qqq
import wandb

In [ ]:
# Log in to your W&B account
wandb.login()

# wandb와 연결

wandb: Currently logged in as: bglepark. Use `wandb login --relogin` to force relogin


True

In [ ]:
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset

@DATASETS.register_module(force=True)
class fishdataset(CocoDataset):
  CLASSES = ('Olive flounder', 'Korea rockfish', 'Red seabream' , 'Black porgy' , 'Rock bream') 
  # 라벨 name을 입력

In [ ]:
config_file = '/content/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = '/content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

In [ ]:
!cd /content/mmdetection; mkdir checkpoints
!wget -O /content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

--2022-05-09 02:11:25--  http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.28
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167287506 (160M) [application/octet-stream]
Saving to: ‘/content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’

/content/mmdetectio 100%[===================>] 159.54M  9.09MB/s    in 19s     

2022-05-09 02:11:45 (8.59 MB/s) - ‘/content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’ saved [167287506/167287506]



In [ ]:
!ls -lia /content/mmdetection/checkpoints

total 163376
5247040 drwxr-xr-x  2 root root      4096 May  7 06:45 .
5243026 drwxr-xr-x 19 root root      4096 May  7 06:45 ..
5247052 -rw-r--r--  1 root root 167287506 Nov  2  2021 faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth


In [ ]:
from mmcv import Config

cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [ ]:
from mmdet.apis import set_random_seed

# dataset에 대한 환경 파라미터 수정. 
cfg.dataset_type = 'fishdataset'
cfg.data_root = '/content/drive/MyDrive/융합프로젝트_낚시/fish_sample_400_100_100_mmdetection/fish_sample/'

# train, val, test dataset에 대한 type, data_root, ann_file, img_prefix 환경 파라미터 수정. 
cfg.data.train.type = 'fishdataset'
cfg.data.train.data_root = '/content/drive/MyDrive/융합프로젝트_낚시/fish_sample_400_100_100_mmdetection/fish_sample/'
cfg.data.train.ann_file = 'train.json'
cfg.data.train.img_prefix = 'images'

cfg.data.val.type = 'fishdataset'
cfg.data.val.data_root = '/content/drive/MyDrive/융합프로젝트_낚시/fish_sample_400_100_100_mmdetection/fish_sample/'
cfg.data.val.ann_file = 'valid.json'
cfg.data.val.img_prefix = 'images'

cfg.data.test.type = 'fishdataset'
cfg.data.test.data_root = '/content/drive/MyDrive/융합프로젝트_낚시/fish_sample_400_100_100_mmdetection/fish_sample/'
cfg.data.test.ann_file = 'test.json'
cfg.data.test.img_prefix = 'images'

# class의 갯수 수정. 
cfg.model.roi_head.bbox_head.num_classes = 5
# pretrained 모델
cfg.load_from = '/content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

# 학습 weight 파일로 로그를 저장하기 위한 디렉토리 설정. 
cfg.work_dir = './tutorial_exps'

# 학습율 변경 환경 파라미터 설정. 
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# CocoDataset의 경우 metric을 bbox로 설정해야 함.(mAP아님. bbox로 설정하면 mAP를 iou threshold를 0.5 ~ 0.95까지 변경하면서 측정)
cfg.evaluation.metric = 'bbox'
cfg.evaluation.interval = 1
cfg.checkpoint_config.interval = 1

# 두번 config를 로드하면 lr_config의 policy가 사라지는 오류로 인하여 설정. 
cfg.lr_config.policy='step'
# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

cfg.runner.max_epochs = 100

cfg.device='cuda'

cfg.train_pipeline[2].img_scale = (640,640)
cfg.test_pipeline[1].img_scale = (640,640)
cfg.data.train.pipeline[2].img_scale = (640,640)
cfg.data.val.pipeline[1].img_scale = (640,640)
cfg.data.test.pipeline[1].img_scale = (640,640)

In [ ]:
print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# train용 Dataset 생성. 
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
print(datasets[0])
# datasets[0].__dict__ 로 모든 self variables의 key와 value값을 볼 수 있음. 
datasets[0].__dict__.keys()


fishdataset Train dataset with number of images 400, and instance counts: 
+--------------------+-------+--------------------+-------+------------------+-------+-----------------+-------+----------------+-------+
| category           | count | category           | count | category         | count | category        | count | category       | count |
+--------------------+-------+--------------------+-------+------------------+-------+-----------------+-------+----------------+-------+
| 0 [Olive flounder] | 80    | 1 [Korea rockfish] | 80    | 2 [Red seabream] | 80    | 3 [Black porgy] | 80    | 4 [Rock bream] | 80    |
+--------------------+-------+--------------------+-------+------------------+-------+-----------------+-------+----------------+-------+


dict_keys(['ann_file', 'data_root', 'img_prefix', 'seg_prefix', 'proposal_file', 'test_mode', 'filter_empty_gt', 'file_client', 'CLASSES', 'coco', 'cat_ids', 'cat2label', 'img_ids', 'data_infos', 'proposals', 'flag', 'pipeline'])

In [ ]:
datasets[0].data_infos

[{'date_captured': '2022-05-06T06:41:17+00:00',
  'file_name': '174268_objt_kr_2020-12-20_11-43-57-66_003_jpg.rf.9278aa18f3b673623875e6395832bbca.jpg',
  'filename': '174268_objt_kr_2020-12-20_11-43-57-66_003_jpg.rf.9278aa18f3b673623875e6395832bbca.jpg',
  'height': 1520,
  'id': 0,
  'license': 1,
  'width': 2704},
 {'date_captured': '2022-05-06T06:41:17+00:00',
  'file_name': '192076_objt_bp_2020-12-22_10-05-49-66_003_jpg.rf.9814385b39493009ed8a878c04f1b47c.jpg',
  'filename': '192076_objt_bp_2020-12-22_10-05-49-66_003_jpg.rf.9814385b39493009ed8a878c04f1b47c.jpg',
  'height': 1520,
  'id': 1,
  'license': 1,
  'width': 2704},
 {'date_captured': '2022-05-06T06:41:17+00:00',
  'file_name': '600852_objt_rb_2021-02-05_09-52-04-33_003_jpg.rf.9a8f71cae6b701740648553b61fa74c1.jpg',
  'filename': '600852_objt_rb_2021-02-05_09-52-04-33_003_jpg.rf.9a8f71cae6b701740648553b61fa74c1.jpg',
  'height': 1520,
  'id': 2,
  'license': 1,
  'width': 2704},
 {'date_captured': '2022-05-06T06:41:17+00:00'

In [ ]:
datasets[0].pipeline

Compose(
    LoadImageFromFile(to_float32=False, color_type='color', channel_order='bgr', file_client_args={'backend': 'disk'})
    LoadAnnotations(with_bbox=True, with_label=True, with_mask=False, with_seg=False, poly2mask=True, poly2mask={'backend': 'disk'})
    Resize(img_scale=[(640, 640)], multiscale_mode=range, ratio_range=None, keep_ratio=True, bbox_clip_border=True)
    RandomFlip(flip_ratio=0.5)
    Normalize(mean=[123.675 116.28  103.53 ], std=[58.395 57.12  57.375], to_rgb=True)
    Pad(size=None, size_divisor=32, pad_to_square=False, pad_val={'img': 0, 'masks': 0, 'seg': 255})
    DefaultFormatBundle(img_to_float=True)
    Collect(keys=['img', 'gt_bboxes', 'gt_labels'], meta_keys=('filename', 'ori_filename', 'ori_shape', 'img_shape', 'pad_shape', 'scale_factor', 'flip', 'flip_direction', 'img_norm_cfg'))
)

In [ ]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

model.CLASSES = datasets[0].CLASSES
print(model.CLASSES)

('Olive flounder', 'Korea rockfish', 'Red seabream', 'Black porgy', 'Rock bream')


In [ ]:
import os.path as osp
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# epochs는 config의 runner 파라미터로 지정됨. 기본 12회 

train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-05-07 07:22:39,707 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-05-07 07:22:39,717 - mmdet - INFO - load checkpoint from local path: /content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2022-05-07 07:22:39,856 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([6, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([6]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([20, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


2022-05-07 07:22:58,201 - mmdet - INFO - Epoch [1][10/200]	lr: 2.500e-03, eta: 7:11:42, time: 1.296, data_time: 1.134, memory: 1414, loss_rpn_cls: 0.0042, loss_rpn_bbox: 0.0025, loss_cls: 0.4661, acc: 90.9863, loss_bbox: 0.1758, loss: 0.6486
2022-05-07 07:23:02,286 - mmdet - INFO - Epoch [1][20/200]	lr: 2.500e-03, eta: 4:43:46, time: 0.409, data_time: 0.265, memory: 1414, loss_rpn_cls: 0.0139, loss_rpn_bbox: 0.0026, loss_cls: 0.1643, acc: 96.0059, loss_bbox: 0.1567, loss: 0.3375
2022-05-07 07:23:06,692 - mmdet - INFO - Epoch [1][30/200]	lr: 2.500e-03, eta: 3:57:57, time: 0.441, data_time: 0.297, memory: 1414, loss_rpn_cls: 0.0110, loss_rpn_bbox: 0.0020, loss_cls: 0.1363, acc: 95.8203, loss_bbox: 0.1621, loss: 0.3114
2022-05-07 07:23:10,230 - mmdet - INFO - Epoch [1][40/200]	lr: 2.500e-03, eta: 3:27:48, time: 0.354, data_time: 0.215, memory: 1414, loss_rpn_cls: 0.0027, loss_rpn_bbox: 0.0028, loss_cls: 0.1237, acc: 95.8691, loss_bbox: 0.1585, loss: 0.2877
2022-05-07 07:23:13,890 - mmdet 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100/100, 5.0 task/s, elapsed: 20s, ETA:     0s

2022-05-07 07:24:32,977 - mmdet - INFO - Evaluating bbox...


...

2022-05-07 08:31:30,718 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.06s).
Accumulating evaluation results...


2022-05-07 08:31:30,832 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.777
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.860
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.836
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.777
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.872
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.872
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.872
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=

DONE (t=0.04s).


learning_rate,████▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▁▆▇▇▇▇▇▇▇█▇█▇▇▇▇▇▇██▇█▇███████▇████▇██▇█
train/loss,█▃▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_bbox,█▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_cls,█▃▂▁▁▂▁▁▂▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▄▅▅▂▄▂▂▂▃▂▂▂▃▃▂▂▄▂▁▁▂▂▂▂▁▂▂▃▂▂▂▁▃▁▂▁▂▂▂
train/loss_rpn_cls,▅▂▂▂▁▂▄▅▁▂▁▁█▂▁▁▂▂▁▁▁▁▃▄▁▁▂▁▁▁▁▃▁▁▂▂▁▁▃▁
val/bbox_mAP,▁▆▆▇████████████████████████████████████
val/bbox_mAP_50,▁███████████████████████████████████████
val/bbox_mAP_75,▁▇██████████████████████████████████████
